### Xeno Underwriting Services: modelling and process explanation 


This notebook will go through the modelling process that Xeno Underwriting services adapts in its products. The code and the basic structure of the following notebook is taken from the research paper listed below, with the relevent changes being explained. The variables used in this model can be verified from the document in the second source listed below.

[1]
A. Noll, R. Salzmann and M.V. Wuthrich, Case Study: French Motor Third-Party Liability Claims (November 8, 2018). doi:10.2139/ssrn.3164764

[2]
"CASdatasets" package supporting document. http://cas.uqam.ca/pub/web/CASdatasets-manual.pdf

In [3]:
%matplotlib inline


# Tweedie regression on insurance claims

This example illustrates the use of Tweedie regression on
the [French Motor Third-Party Liability Claims dataset](https://www.openml.org/d/41214), and is inspired by an R tutorial [1]_.

In this dataset, each sample corresponds to an insurance policy, i.e. a
contract within an insurance company and an individual (policyholder).
Available features include driver age, vehicle age, vehicle power, etc.

A few definitions: a *claim* is the request made by a policyholder to the
insurer to compensate for a loss covered by the insurance. The *claim amount*
is the amount of money that the insurer must pay. The *exposure* is the
duration of the insurance coverage of a given policy, in years.

Here our goal is to predict the expected
value, i.e. the mean, of the total claim amount per exposure unit also
referred to as the pure premium.

There are several possibilities to do that, two of which are:

1. Model the number of claims with a Poisson distribution, and the average
   claim amount per claim, also known as severity, as a Gamma distribution
   and multiply the predictions of both in order to get the total claim
   amount.
2. Model the total claim amount per exposure directly, typically with a Tweedie
   distribution of Tweedie power $p \in (1, 2)$.

In this example we will illustrate both approaches. We start by defining a few
helper functions for loading the data and visualizing results.




In [4]:
# Authors: Christian Lorentzen <lorentzen.ch@gmail.com>
#          Roman Yurchak <rth.yurchak@gmail.com>
#          Olivier Grisel <olivier.grisel@ensta.org>
# License: BSD 3 clause

In [5]:
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_tweedie_deviance
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


def load_mtpl2(n_samples=None):
    """Fetch the French Motor Third-Party Liability Claims dataset.

    Parameters
    ----------
    n_samples: int, default=None
      number of samples to select (for faster run time). Full dataset has
      678013 samples.
    """
    # freMTPL2freq dataset from https://www.openml.org/d/41214
    df_freq = fetch_openml(data_id=41214, as_frame=True, parser="pandas").data
    df_freq["IDpol"] = df_freq["IDpol"].astype(int)
    df_freq.set_index("IDpol", inplace=True)

    # freMTPL2sev dataset from https://www.openml.org/d/41215
    df_sev = fetch_openml(data_id=41215, as_frame=True, parser="pandas").data

    # sum ClaimAmount over identical IDs
    df_sev = df_sev.groupby("IDpol").sum()

    df = df_freq.join(df_sev, how="left")
    df["ClaimAmount"].fillna(0, inplace=True)

    # unquote string fields
    for column_name in df.columns[df.dtypes.values == object]:
        df[column_name] = df[column_name].str.strip("'")
    return df.iloc[:n_samples]


def plot_obs_pred(
    df,
    feature,
    weight,
    observed,
    predicted,
    y_label=None,
    title=None,
    ax=None,
    fill_legend=False,
):
    """Plot observed and predicted - aggregated per feature level.

    Parameters
    ----------
    df : DataFrame
        input data
    feature: str
        a column name of df for the feature to be plotted
    weight : str
        column name of df with the values of weights or exposure
    observed : str
        a column name of df with the observed target
    predicted : DataFrame
        a dataframe, with the same index as df, with the predicted target
    fill_legend : bool, default=False
        whether to show fill_between legend
    """
    # aggregate observed and predicted variables by feature level
    df_ = df.loc[:, [feature, weight]].copy()
    df_["observed"] = df[observed] * df[weight]
    df_["predicted"] = predicted * df[weight]
    df_ = (
        df_.groupby([feature])[[weight, "observed", "predicted"]]
        .sum()
        .assign(observed=lambda x: x["observed"] / x[weight])
        .assign(predicted=lambda x: x["predicted"] / x[weight])
    )

    ax = df_.loc[:, ["observed", "predicted"]].plot(style=".", ax=ax)
    y_max = df_.loc[:, ["observed", "predicted"]].values.max() * 0.8
    p2 = ax.fill_between(
        df_.index,
        0,
        y_max * df_[weight] / df_[weight].values.max(),
        color="g",
        alpha=0.1,
    )
    if fill_legend:
        ax.legend([p2], ["{} distribution".format(feature)])
    ax.set(
        ylabel=y_label if y_label is not None else None,
        title=title if title is not None else "Train: Observed vs Predicted",
    )


def score_estimator(
    estimator,
    X_train,
    X_test,
    df_train,
    df_test,
    target,
    weights,
    tweedie_powers=None,
):
    """Evaluate an estimator on train and test sets with different metrics"""

    metrics = [
        ("D² explained", None),  # Use default scorer if it exists
        ("mean abs. error", mean_absolute_error),
        ("mean squared error", mean_squared_error),
    ]
    if tweedie_powers:
        metrics += [
            (
                "mean Tweedie dev p={:.4f}".format(power),
                partial(mean_tweedie_deviance, power=power),
            )
            for power in tweedie_powers
        ]

    res = []
    for subset_label, X, df in [
        ("train", X_train, df_train),
        ("test", X_test, df_test),
    ]:
        y, _weights = df[target], df[weights]
        for score_label, metric in metrics:
            if isinstance(estimator, tuple) and len(estimator) == 2:
                # Score the model consisting of the product of frequency and
                # severity models.
                est_freq, est_sev = estimator
                y_pred = est_freq.predict(X) * est_sev.predict(X)
            else:
                y_pred = estimator.predict(X)

            if metric is None:
                if not hasattr(estimator, "score"):
                    continue
                score = estimator.score(X, y, sample_weight=_weights)
            else:
                score = metric(y, y_pred, sample_weight=_weights)

            res.append({"subset": subset_label, "metric": score_label, "score": score})

    res = (
        pd.DataFrame(res)
        .set_index(["metric", "subset"])
        .score.unstack(-1)
        .round(4)
        .loc[:, ["train", "test"]]
    )
    return res

## Loading datasets, basic feature extraction and target definitions

We construct the freMTPL2 dataset by joining the freMTPL2freq table,
containing the number of claims (``ClaimNb``), with the freMTPL2sev table,
containing the claim amount (``ClaimAmount``) for the same policy ids
(``IDpol``).

#### Filtering and Correcting Data:

The code filters out claims with zero amounts and sets their corresponding claim count (ClaimNb) to zero. This is done to ensure that the severity model, which requires strictly positive target values, can be applied.

Unreasonable observations are corrected to mitigate potential data errors. Specifically, ClaimNb, Exposure, and ClaimAmount are clipped to upper limits of 4, 1, and 200,000, respectively. This prevents exceptionally large claim amounts and exposure values.


#### Data Transformation:

ColumnTransformer is then used to define the following set of trasnformers:

- KBinsDiscretizer is used to discretize the "VehAge" and "DrivAge" features into 10 bins each, resulting in binned numeric features.

- OneHotEncoder is applied to "VehBrand", "VehPower", "VehGas",  to convert categorical variables into one-hot encoded features.


- "Density" is log-scaled using FunctionTransformer with a logarithm function.

#### Feature Extraction

The "PurePremium" feature represents the expected total claim amount per unit of exposure for each policyholder in the insurance company's portfolio. It is computed by dividing the "ClaimAmount" by the "Exposure" in the dataset.

#### Dropping Redundant Variables:

The "Area", "Region", and "BonusMalus" columns are dropped from the training data as they are only concerned with French areas and insurance rating, while this MVP is directed towards clients in different parts of Europe. 




In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


df = load_mtpl2()

# Correct for unreasonable observations (that might be data error)
# and a few exceptionally large claim amounts
df["ClaimNb"] = df["ClaimNb"].clip(upper=4)
df["Exposure"] = df["Exposure"].clip(upper=1)
df["ClaimAmount"] = df["ClaimAmount"].clip(upper=200000)

log_scale_transformer = make_pipeline(FunctionTransformer(func=np.log), StandardScaler())

column_trans = ColumnTransformer(
    [("binned_numeric",KBinsDiscretizer(n_bins=10, subsample=int(2e5), random_state=0),["VehAge", "DrivAge"],),
    ("onehot_categorical",OneHotEncoder(), ["VehBrand", "VehPower", "VehGas"],),
    ("log_scaled_numeric", log_scale_transformer, ["Density"]),], remainder="drop",)

X = column_trans.fit_transform(df)

# Pure Premium 
df["PurePremium"] = df["ClaimAmount"] / df["Exposure"]


#### dropping redundant variables for Xeno's applications
df = df.drop(['Area', 'Region', 'BonusMalus'], axis=1)
### 

##Splitting the dataframe to train and test data 
df_train, df_test, X_train, X_test = train_test_split(df, X, random_state=0)
pd.set_option('display.max_rows', None)

####


4


### Variables 

PurePremium: the expected total claim amount per unit of exposure for each policyholder in their portfolio. 

ClaimNb: The number of claims.

Exposure: The number of policy years.

VehPower: The vehicle power (as "factor") from least powerful "P2" to most powerful car "P15".

VehAge: The vehicle age, in years.

DrivAge: The age of the policyholder.

VehBrand: The car brand divided in the following groups: A- Renaut Nissan and Citroen, B- Volkswagen, Audi, Skoda and Seat, C- Opel, General Motors and Ford, D- Fiat, E- Mercedes Chrysler
and BMW, F- Japanese (except Nissan) and Korean, G- other.

VehGas: The car gas, Diesel or regular (as "factor").

Density: The density of inhabitants (number of inhabitants per km2) in the city the driver of the
car lives in.

ClaimAmount: Total previous claim amount of the guarantee.

 








## Pure Premium Modeling via a  TweedieRegressor
We can model the total loss with a unique
Compound Poisson Gamma generalized linear model (with a log link function).
This model is a special case of the Tweedie GLM with a "power" parameter
$p \in (1, 2)$. Here, we fix apriori the `power` parameter of the
Tweedie model to some arbitrary value (1.9) in the valid range. Ideally one
would select this value via grid-search by minimizing the negative
log-likelihood of the Tweedie model, but unfortunately the current
implementation does not allow for this (yet).

We will compare the performance of both approaches.
To quantify the performance of both models, one can compute
the mean deviance of the train and test data assuming a Compound
Poisson-Gamma distribution of the total claim amount. This is equivalent to
a Tweedie distribution with a `power` parameter between 1 and 2.

The :func:`sklearn.metrics.mean_tweedie_deviance` depends on a `power`
parameter. As we do not know the true value of the `power` parameter, we here
compute the mean deviances for a grid of possible values, and compare the
models side by side, i.e. we compare them at identical values of `power`.
Ideally, we hope that one model will be consistently better than the other,
regardless of `power`.



In [30]:
from sklearn.linear_model import TweedieRegressor
from joblib import dump, load
import pickle


glm_pure_premium = TweedieRegressor(power=1.9, alpha=0.1, solver="newton-cholesky")
glm_pure_premium_1= glm_pure_premium.fit(
    X_train, df_train["PurePremium"], sample_weight=df_train["Exposure"]
)

tweedie_powers = [1.5, 1.7, 1.8, 1.9, 1.99, 1.999, 1.9999]



scores_glm_pure_premium = score_estimator(
    glm_pure_premium,
    X_train,
    X_test,
    df_train,
    df_test,
    target="PurePremium",
    weights="Exposure",
    tweedie_powers=tweedie_powers,)

scores = pd.concat(
    [scores_glm_pure_premium],
    axis=1,
    sort=True,
    keys=("Product Model", "TweedieRegressor"),)

print("Evaluation of the Product Model and the Tweedie Regressor on target PurePremium")
with pd.option_context("display.expand_frame_repr", False):
    print(scores)

#### exctracting the model 
with open('model.pkl', 'wb') as f:
    pickle.dump(glm_pure_premium_1, f)
#### 
y_pred = glm_pure_premium_1.predict(X_test)


Evaluation of the Product Model and the Tweedie Regressor on target PurePremium
                          Product Model              
subset                            train          test
metric                                               
D² explained               1.690000e-02  1.420000e-02
mean Tweedie dev p=1.5000  7.640770e+01  7.640880e+01
mean Tweedie dev p=1.7000  3.682880e+01  3.692270e+01
mean Tweedie dev p=1.8000  3.037600e+01  3.045390e+01
mean Tweedie dev p=1.9000  3.382120e+01  3.387830e+01
mean Tweedie dev p=1.9900  2.015347e+02  2.015587e+02
mean Tweedie dev p=1.9990  1.914538e+03  1.914387e+03
mean Tweedie dev p=1.9999  1.904747e+04  1.904558e+04
mean abs. error            2.739865e+02  2.731249e+02
mean squared error         3.295505e+07  3.213056e+07
[ 87.1174128   87.93388079  75.93685879 ... 117.15437623  74.95634597
  70.43728507]
